In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['Data_Dictionary.xlsx', 'new_merchant_transactions.csv', 'test.csv', 'merchants.csv', 'historical_transactions.csv', 'train.csv', 'sample_submission.csv']


In [45]:
import featuretools

In [46]:
new_transactions = pd.read_csv('../input/new_merchant_transactions.csv')
historical_transactions = pd.read_csv('../input/historical_transactions.csv')

def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [47]:
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df
#_________________________________________
train = read_data('../input/train.csv')
test = read_data('../input/test.csv')

target = train['target']
del train['target']

In [48]:
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])

In [49]:
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 'category_3'])

In [50]:
merchants = pd.read_csv('../input/merchants.csv')


In [51]:
import gc
alldata = train.append(test,ignore_index=True)

del train
del test

gc.collect()

570

In [52]:
def binarize_merchants(df):
    for col in ['category_1','category_4']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

In [53]:
unique_merchants = merchants.set_index("merchant_id").groupby(level=0).first().reset_index()
unique_merchants = pd.get_dummies(unique_merchants,columns=['most_recent_sales_range','most_recent_purchases_range'])
unique_merchants = binarize_merchants(unique_merchants)

In [54]:
unique_merchants.head()
del merchants
gc.collect()

77

In [55]:
unique_merchants.columns.tolist()

['merchant_id',
 'merchant_group_id',
 'merchant_category_id',
 'subsector_id',
 'numerical_1',
 'numerical_2',
 'category_1',
 'avg_sales_lag3',
 'avg_purchases_lag3',
 'active_months_lag3',
 'avg_sales_lag6',
 'avg_purchases_lag6',
 'active_months_lag6',
 'avg_sales_lag12',
 'avg_purchases_lag12',
 'active_months_lag12',
 'category_4',
 'city_id',
 'state_id',
 'category_2',
 'most_recent_sales_range_A',
 'most_recent_sales_range_B',
 'most_recent_sales_range_C',
 'most_recent_sales_range_D',
 'most_recent_sales_range_E',
 'most_recent_purchases_range_A',
 'most_recent_purchases_range_B',
 'most_recent_purchases_range_C',
 'most_recent_purchases_range_D',
 'most_recent_purchases_range_E']

In [56]:
all_transactions = historical_transactions.append(new_transactions)
# all_transactions["mer_cus_id"] = all_transactions["merchant_id"] + all_transactions["card_id"] + all_transactions["purchase_date"]

In [57]:
def unique_field(x):
    global count
    count = count + 1 
    return count

count = 0 
    
all_transactions["u_id"] = all_transactions["merchant_id"].apply(unique_field)

In [58]:
len(set(all_transactions["u_id"].values.tolist()))

31075392

In [59]:
all_transactions.count()

authorized_flag         31075392
card_id                 31075392
city_id                 31075392
category_1              31075392
installments            31075392
merchant_category_id    31075392
merchant_id             30910695
month_lag               31075392
purchase_amount         31075392
purchase_date           31075392
state_id                31075392
subsector_id            31075392
category_2_1.0          31075392
category_2_2.0          31075392
category_2_3.0          31075392
category_2_4.0          31075392
category_2_5.0          31075392
category_3_A            31075392
category_3_B            31075392
category_3_C            31075392
u_id                    31075392
dtype: int64

In [60]:
unique_merchants.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
0,M_ID_000025127f,14602,80,37,-0.057471,-0.057471,0,1.29,1.880342,3,1.26,2.228632,6,1.08,2.089744,12,1,69,9,1.0,0,0,0,0,1,0,0,0,0,1
1,M_ID_0000699140,19420,87,27,-0.057471,-0.057471,0,0.99,0.997832,3,1.04,1.002168,6,1.24,1.154878,12,0,48,9,1.0,0,0,0,1,0,0,0,0,1,0
2,M_ID_00006a5552,52848,178,29,-0.057471,-0.057471,0,1.21,1.000000,3,1.18,1.000000,6,1.22,1.035714,12,0,64,15,1.0,0,0,0,0,1,0,0,0,0,1
3,M_ID_000087311e,44655,206,1,-0.057471,-0.057471,1,0.93,1.314815,3,0.95,1.300926,6,0.78,1.261574,12,1,-1,-1,NaN,0,0,0,1,0,0,0,0,0,1
4,M_ID_0000ab0b2d,35761,367,16,-0.057471,-0.057471,0,1.05,1.089897,3,0.99,1.057279,6,1.06,1.135640,12,0,-1,9,1.0,0,0,0,1,0,0,0,0,1,0


In [61]:
unique_merchants["category_1"].value_counts()

0    327618
1      7015
Name: category_1, dtype: int64

In [62]:
import featuretools.variable_types

transactions_variable_types = {
'city_id' : featuretools.variable_types.Categorical,
'merchant_category_id' : featuretools.variable_types.Categorical,
'state_id' : featuretools.variable_types.Categorical,
'subsector_id' : featuretools.variable_types.Categorical
}

merchants_variable_types = {
    'merchant_group_id' : featuretools.variable_types.Categorical,
    'merchant_category_id' : featuretools.variable_types.Categorical,
    'subsector_id' : featuretools.variable_types.Categorical,
    'city_id' : featuretools.variable_types.Categorical,
    'state_id' : featuretools.variable_types.Categorical,
    'category_2' : featuretools.variable_types.Categorical

}

In [63]:
import featuretools as ft

card_score = ft.EntitySet(id = 'cs')
card_score.entity_from_dataframe(
    entity_id = 'transactions'
    , dataframe = all_transactions
    , index = 'u_id'
    ,variable_types= transactions_variable_types)

card_score.entity_from_dataframe(entity_id = 'card_score', dataframe = alldata, index = 'card_id')

card_score.entity_from_dataframe(
    entity_id = 'merchants',
    dataframe = unique_merchants, 
    index = 'merchant_id',
    variable_types = merchants_variable_types
    )

Entityset: cs
  Entities:
    transactions [Rows: 31075392, Columns: 21]
    card_score [Rows: 325540, Columns: 6]
    merchants [Rows: 334633, Columns: 30]
  Relationships:
    No relationships

In [64]:
relationship_card_transactions = ft.Relationship(card_score['card_score']['card_id'], card_score['transactions']['card_id'])
relationship_merchant_transactions = ft.Relationship(card_score['merchants']['merchant_id'], card_score['transactions']['merchant_id'])
card_score.add_relationship(relationship_card_transactions)
card_score.add_relationship(relationship_merchant_transactions)

Entityset: cs
  Entities:
    transactions [Rows: 31075392, Columns: 21]
    card_score [Rows: 325540, Columns: 6]
    merchants [Rows: 334633, Columns: 30]
  Relationships:
    transactions.card_id -> card_score.card_id
    transactions.merchant_id -> merchants.merchant_id

In [76]:
from dask.distributed import LocalCluster

cluster = LocalCluster(processes=False)

In [ ]:
features, feature_names = ft.dfs(entityset=card_score,target_entity='card_score',max_depth=4,verbose=True,chunk_size=10,dask_kwargs={'cluster': cluster})

Built 801 features
2018-12-09 22:15:33,755 featuretools.computational_backend - WARNING    Worker memory is between 1 to 2 times the memory size of the EntitySet. If errors occur that do not occur with n_jobs equals 1, this may be the cause.  See https://docs.featuretools.com/guides/parallel.html for more information.


distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/8 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/9 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/10 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/11 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/12 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/13 remote=inproc://192.168.1.2/15359/1>


EntitySet scattered to workers in 29.845 seconds


distributed.utils - ERROR - '<' not supported between instances of 'NoneType' and 'tuple'
Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/utils.py", line 646, in log_errors
    yield
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/bokeh/scheduler.py", line 1126, in graph_doc
    graph = GraphPlot(scheduler, sizing_mode='stretch_both')
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/bokeh/scheduler.py", line 609, in __init__
    self.layout = GraphLayout(scheduler)
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/diagnostics/graph_layout.py", line 38, in __init__
    priority=priority)
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/diagnostics/graph_layout.py", line 42, in update_graph
    stack = sorted(dependencies, key=priority.get, reverse=True)
TypeError: '<' not support

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/34 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/35 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/36 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/37 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/38 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/39 remote=inproc://192.168.1.2/15359/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.1.2/15359/40 remote=inproc://192.168.1.2/15359/1>






Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/32554 chunks

In [ ]:
print(feature_names)

In [ ]:
features.head()

In [ ]:
features.to_csv("features_8.csv")